In [ ]:
!pip install fastapi
!pip install colabcode
!pip install tweepy

In [ ]:
from os import getenv
import json
import uuid

import tweepy

from fastapi import FastAPI

In [ ]:
config = {
  "CONSUMER_KEY": "",
  "CONSUMER_SECRET": "",
  "ACCESS_TOKEN": "",
  "ACCESS_TOKEN_SECRET": ""
}

In [ ]:
# Authenticate to Twitter
def api_auth():
  auth = tweepy.OAuthHandler(config['CONSUMER_KEY'], config['CONSUMER_SECRET'])
  auth.set_access_token(config['ACCESS_TOKEN'], config['ACCESS_TOKEN_SECRET'])

  # Create API object
  api = tweepy.API(auth, wait_on_rate_limit=True,
      wait_on_rate_limit_notify=True)
  try:
      api.verify_credentials()
      print("Authentication OK")
      return api
  except:
      print("Error during authentication")
      return None

In [ ]:
# Create a tweet
def post_message(api,text):
  return api.update_status(text)

# Get timeline tweets
def get_timeline_messages(api,count=100):
  tweets = {}
  timeline = api.home_timeline(count)
  for tweet in timeline:
     tweets[tweet.id] = {"username": tweet.user.name, "message" : tweet.text, "date": ""}
  return tweets

# Check mentions sent to @
def check_mentions(api, keywords, since_id):
    logger.info("Retrieving mentions")
    new_since_id = since_id
    for tweet in tweepy.Cursor(api.mentions_timeline,
        since_id=since_id).items():
        new_since_id = max(tweet.id, new_since_id)
        if tweet.in_reply_to_status_id is not None:
            continue
        if any(keyword in tweet.text.lower() for keyword in keywords):
            logger.info(f"Answering to {tweet.user.name}")

            if not tweet.user.following:
                tweet.user.follow()

            api.update_status(
                status="Please reach us via DM",
                in_reply_to_status_id=tweet.id,
            )
    return new_since_id

In [ ]:
app = FastAPI()

@app.on_event("startup")
def load_model():
    global query, model, transformation
    #fpickle = pickle.load(open("model_tree.pkl", "rb"))
    #model = fpickle['model']
    #transformation = fpickle['transformation']

@app.get('/')
def index():
    return {'message': 'This is the homepage of the API '}

@app.get('/tweet/{item_code}')
def get_data_by_zip(item_code: int):
    query = "select top 1 * from c where c.municipio_codigo = {}".format(str(item_code))
    print(query)
    result = get_query_items(query)
    return result